In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# To remove the scientific notation from numpy arrays
np.set_printoptions(suppress=True)

In [ ]:
premonsoon_data = pd.read_csv('/content/ALL_DCF.csv')

In [ ]:
premonsoon_data.head(2)

In [ ]:
premonsoon_data.corr(method ='pearson')

In [ ]:
# Separate Target Variable and Predictor Variables
TargetVariable=['Output']
Predictors=['Smap_S1','DC','TWI', 'Filled_DEM', 'NDVI_L9', 'LST', 'SMI','Sigma0-VV','Sigma0-VH','TM','HM']

In [ ]:
X = premonsoon_data[Predictors].values
Y = premonsoon_data[TargetVariable].values

### Sandardization of data ###
from sklearn.preprocessing import StandardScaler
PredictorScaler=StandardScaler()
TargetVarScaler=StandardScaler()

# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X)
TargetVarScalerFit=TargetVarScaler.fit(Y)

# Generating the standardized values of X and y
x=PredictorScalerFit.transform(X)
y=TargetVarScalerFit.transform(Y)

In [ ]:
# Split the data into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=5)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
!pip install tensorflow
!pip install keras

In [ ]:
# importing the libraries
from keras.models import Sequential
from keras.layers import Dense
from sklearn import metrics
from math import sqrt

# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):

    # Defining the list of hyper parameters to try
    batch_size_list=[5, 10, 15, 20]
    epoch_list  =   [5, 10, 50, 100]

    import pandas as pd
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'Accuracy'])

    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber+=1
            # create ANN model
            model = Sequential()
            # Defining the first layer of the model
            model.add(Dense(units=20, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))

            # Defining the Second layer of the model
            model.add(Dense(units=25, kernel_initializer='normal', activation='relu'))

            model.add(Dense(units=20, kernel_initializer='normal', activation='relu'))

            # The output neuron is a single fully connected node
            # Since we will be predicting a single number
            model.add(Dense(1, kernel_initializer='normal'))

            # Compiling the model
            model.compile(loss='mean_squared_error', optimizer='adam')

            # Fitting the ANN to the Training set
            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)

            # printing the results of the current iteration
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'R2', metrics.r2_score(y_test, model.predict(X_test)), 'RMSE', sqrt(metrics.mean_squared_error(y_test, model.predict(X_test))))

            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), metrics.r2_score(y_test, model.predict(X_test)), sqrt(metrics.mean_squared_error(y_test, model.predict(X_test)))]],
                                                                    columns=['TrialNumber', 'Parameters', 'R2', 'RMSE'] ))
    return(SearchResultsData)


######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

In [ ]:
# 3 Parameters: batch_size: 5 - epochs: 50 R2 0.6805536606970652 RMSE 0.6088032779263932